<a href="https://colab.research.google.com/github/busiri/busil/blob/main/5%EB%8B%A8%EA%B3%84_(%EB%AA%A8%EB%8D%B8%EB%A7%81).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier
from sklearn.preprocessing import RobustScaler, KBinsDiscretizer, PolynomialFeatures
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.pipeline       import Pipeline
from imblearn.ensemble       import BalancedRandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_predict,train_test_split,cross_val_score
from sklearn.metrics         import (
    classification_report,
    average_precision_score,
    recall_score,
    precision_score,
    f1_score
)
import platform
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from joblib import parallel_backend


# 한글 폰트 설정 (예: 맑은 고딕)
# 폰트 설치
!sudo apt-get install -y fonts-nanum

# 설치된 Nanum 폰트 경로 확인
font_dirs = ['/usr/share/fonts/truetype/nanum/']
font_files = fm.findSystemFonts(fontpaths=font_dirs)

# 폰트 매니저에 폰트 추가
for font_file in font_files:
    fm.fontManager.addfont(font_file)

for f in fm.findSystemFonts(fontpaths=None, fontext='ttf'):
    if 'Nanum' in f:
        print(f)
# 폰트 이름 확인
nanum_font = fm.FontProperties(fname=font_files[0]).get_name()

# 폰트 설정
plt.rcParams['font.family'] = nanum_font
plt.rcParams['axes.unicode_minus'] = False
mpl.rcParams['font.family'] = nanum_font
mpl.rcParams['axes.unicode_minus'] = False

font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_prop = fm.FontProperties(fname=font_path)

plt.rcParams['font.family'] = font_prop.get_name()
plt.rcParams['axes.unicode_minus'] = False

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (6,158 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 126281 files and dire

### 데이터 불러오기

In [3]:
#도입기
train_df = pd.read_csv("/content/도입_final_파생추가.csv")
train_df.drop('회사명',axis=1,inplace=True)
test_df = pd.read_csv("/content/도입기_test.csv")
test_df = test_df[train_df.columns]

# #성장기
# train_df = pd.read_csv("/content/성장_final_파생추가.csv")
# train_df.drop('회사명',axis=1,inplace=True)
# test_df = pd.read_csv("/content/성장기_test.csv")
# test_df = test_df[train_df.columns]

# #성숙기
# train_df = pd.read_csv("/content/성숙_final_파생추가.csv")
# train_df.drop('회사명',axis=1,inplace=True)
# test_df = pd.read_csv("/content/성숙기_test.csv")
# test_df = test_df[train_df.columns]

# #쇠퇴기
# train_df = pd.read_csv("/content/쇠퇴_final_파생추가.csv")
# train_df.drop('회사명',axis=1,inplace=True)
# test_df = pd.read_csv("/content/쇠퇴기_test.csv")
# test_df = test_df[train_df.columns]

In [4]:
# 기업규모명, 통계청 한국표준산업분류 11차(대분류) 라벨인코딩
from sklearn.preprocessing import LabelEncoder

encoder1 = LabelEncoder()
encoder2 = LabelEncoder()
train_df['기업규모명'] = encoder1.fit_transform(train_df['기업규모명'])
test_df['기업규모명'] = encoder1.transform(test_df['기업규모명'])

train_df['통계청 한국표준산업분류 11차(대분류)'] = encoder2.fit_transform(train_df['통계청 한국표준산업분류 11차(대분류)'])
test_df['통계청 한국표준산업분류 11차(대분류)'] = encoder2.transform(test_df['통계청 한국표준산업분류 11차(대분류)'])

,회사명,기업규모명,통계청 한국표준산업분류 11차(대분류)
0,제일스텐철강주식회사,중소기업,도매 및 소매업
1,엘앤케이씨티,중소기업,운수 및 창고업
2,(주)나비디벨롭먼트,중소기업,건설업
3,주식회사엔이티,중소기업,"전기, 가스, 증기 및 공기조절 공급업"
4,애플디앤씨(주),중소기업,부동산업
...,...,...,...
7871,주식회사강림C.S.P,중견기업,제조업
7872,(주)안산이에스,중소기업,"수도, 하수 및 폐기물 처리, 원료 재생업"
7873,주식회사남경플러스,중소기업,부동산업
7874,주식회사보령파트너스,중소기업,"전문, 과학 및 기술 서비스업"


In [ ]:
df_tr, y_train = train_df.drop('부실여부',axis=1), train_df['부실여부']
df_te, y_test = test_df.drop('부실여부',axis=1), test_df['부실여부']

### 모델링 : 오버샘플링 진행하여 학습

### 모델1 : Balanced RF

In [ ]:
# ─────────────────────────────────────────────────────────────────
# 1) Balanced RF
# ─────────────────────────────────────────────────────────────────
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
SM = BorderlineSMOTE(sampling_strategy=0.3, kind='borderline-1', random_state=42)

# ------------------------------------------------------------------
# 파이프라인 정의:
#   1) SMOTE
#   2) BalancedRF로 feature_importances_ 학습
#   3) BalancedRF로 최종 예측
# ------------------------------------------------------------------
pipe_brf_top50 = Pipeline([
    ('smote',   SM),
    ('feat_sel', SelectFromModel(
        estimator=BalancedRandomForestClassifier(
            n_estimators=100, random_state=42
        ),

        threshold=-np.inf,
        prefit=False
    )),
    ('clf',     BalancedRandomForestClassifier(
        n_estimators=100, random_state=42
    ))
])

# ------------------------------------------------------------------
# 평가 함수 (5-fold CV + test)
# ------------------------------------------------------------------
def evaluate(pipe, X_tr, y_tr, X_te, y_te, thresh=0.5):
    # threading 백엔드로 병렬 실행
    with parallel_backend('threading', n_jobs=-1):
        y_val_proba = cross_val_predict(
            pipe, X_tr, y_tr,
            cv=cv,
            method='predict_proba',
            n_jobs=-1
        )[:,1]
    val_pr  = average_precision_score(y_tr, y_val_proba)
    y_val_pred = (y_val_proba >= thresh).astype(int)
    val_rec = recall_score(y_tr, y_val_pred)
    val_pre = precision_score(y_tr, y_val_pred)

    print("===== Validation (5-fold) =====")
    print(f"PR AUC   : {val_pr:.4f}")
    print(f"Recall   : {val_rec:.4f}")
    print(f"Precision: {val_pre:.4f}")
    print(classification_report(y_tr, y_val_pred, digits=4))

    # 전체 학습 → 테스트
    pipe.fit(X_tr, y_tr)
    y_test_proba = pipe.predict_proba(X_te)[:,1]
    y_test_pred  = (y_test_proba >= thresh).astype(int)
    test_pr  = average_precision_score(y_te, y_test_proba)
    test_rec = recall_score(y_te, y_test_pred)
    test_pre = precision_score(y_te, y_test_pred)
    test_f1 = f1_score(y_te,y_test_pred)

    print("===== Test =====")
    print(f"PR AUC   : {test_pr:.4f}")
    print(f"Recall   : {test_rec:.4f}")
    print(f"Precision: {test_pre:.4f}")
    print(f"f1_score: {test_f1:.4f}")
    print(classification_report(y_te, y_test_pred, digits=4))

# ------------------------------------------------------------------
# 실행
# ------------------------------------------------------------------
evaluate(pipe_brf_top50, df_tr, y_train, df_te, y_test, thresh=0.5)

===== Validation (5-fold) =====
PR AUC   : 0.3956
Recall   : 0.5905
Precision: 0.3560
              precision    recall  f1-score   support

           0     0.9840    0.9594    0.9715     16563
           1     0.3560    0.5905    0.4442       630

    accuracy                         0.9459     17193
   macro avg     0.6700    0.7749    0.7079     17193
weighted avg     0.9610    0.9459    0.9522     17193

===== Test =====
PR AUC   : 0.4164
Recall   : 0.6646
Precision: 0.3832
f1_score: 0.4861
              precision    recall  f1-score   support

           0     0.9868    0.9592    0.9728      4141
           1     0.3832    0.6646    0.4861       158

    accuracy                         0.9484      4299
   macro avg     0.6850    0.8119    0.7295      4299
weighted avg     0.9646    0.9484    0.9549      4299



### 모델2 : 로지스틱 회귀

In [ ]:
# ─────────────────────────────────────────────────────────────────
# 2) Logistic Regression
# ─────────────────────────────────────────────────────────────────
from sklearn.linear_model import LogisticRegression

pipe_lr = Pipeline([
    ('smote',   SM),
    ('feat_sel', SelectFromModel(
        estimator=BalancedRandomForestClassifier(n_estimators=100, random_state=42),
         threshold=-np.inf, prefit=False
    )),
    ('clf',     LogisticRegression(solver='liblinear', random_state=42))
])

print("### Logistic Regression ###")
evaluate(pipe_lr, df_tr, y_train, df_te, y_test, thresh=0.5)

### Logistic Regression ###
===== Validation (5-fold) =====
PR AUC   : 0.3128
Recall   : 0.6016
Precision: 0.3044
              precision    recall  f1-score   support

           0     0.9843    0.9477    0.9656     16563
           1     0.3044    0.6016    0.4043       630

    accuracy                         0.9350     17193
   macro avg     0.6443    0.7747    0.6850     17193
weighted avg     0.9593    0.9350    0.9451     17193

===== Test =====
PR AUC   : 0.3495
Recall   : 0.6392
Precision: 0.3217
              precision    recall  f1-score   support

           0     0.9857    0.9486    0.9668      4141
           1     0.3217    0.6392    0.4280       158

    accuracy                         0.9372      4299
   macro avg     0.6537    0.7939    0.6974      4299
weighted avg     0.9613    0.9372    0.9470      4299



### 모델3 : LightGBM

In [ ]:
import numpy as np
import lightgbm as lgb
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.pipeline import Pipeline
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import (
    classification_report,
    average_precision_score,
    recall_score,
    precision_score
)
from joblib import parallel_backend

# 준비된 데이터: df_tr, df_te, y_train, y_test

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
SM = BorderlineSMOTE(sampling_strategy=0.3, kind='borderline-1', random_state=42)

# LGBM 파이프라인 정의
pipe_lgbm_top50 = Pipeline([
    ('smote', SM),
    ('feat_sel', SelectFromModel(
        estimator=BalancedRandomForestClassifier(
            n_estimators=100, random_state=42
        ),

        threshold=-np.inf,
        prefit=False
    )),
    ('clf', lgb.LGBMClassifier(
        n_estimators=100,
        learning_rate=0.05,
        class_weight='balanced',
        random_state=42
    ))
])

# 평가 함수 그대로 사용
def evaluate(pipe, X_tr, y_tr, X_te, y_te, thresh=0.5):
    with parallel_backend('threading', n_jobs=-1):
        y_val_proba = cross_val_predict(
            pipe, X_tr, y_tr,
            cv=cv,
            method='predict_proba',
            n_jobs=-1
        )[:, 1]
    val_pr  = average_precision_score(y_tr, y_val_proba)
    y_val_pred = (y_val_proba >= thresh).astype(int)
    val_rec = recall_score(y_tr, y_val_pred)
    val_pre = precision_score(y_tr, y_val_pred)

    print("===== Validation (5-fold) =====")
    print(f"PR AUC   : {val_pr:.4f}")
    print(f"Recall   : {val_rec:.4f}")
    print(f"Precision: {val_pre:.4f}")
    print(classification_report(y_tr, y_val_pred, digits=4))

    pipe.fit(X_tr, y_tr)
    y_test_proba = pipe.predict_proba(X_te)[:, 1]
    y_test_pred  = (y_test_proba >= thresh).astype(int)
    test_pr  = average_precision_score(y_te, y_test_proba)
    test_rec = recall_score(y_te, y_test_pred)
    test_pre = precision_score(y_te, y_test_pred)

    print("===== Test =====")
    print(f"PR AUC   : {test_pr:.4f}")
    print(f"Recall   : {test_rec:.4f}")
    print(f"Precision: {test_pre:.4f}")
    print(classification_report(y_te, y_test_pred, digits=4))

# 실행
evaluate(pipe_lgbm_top50, df_tr, y_train, df_te, y_test, thresh=0.5)


[LightGBM] [Info] Number of positive: 3975, number of negative: 13250
[LightGBM] [Info] Number of positive: 3975, number of negative: 13250
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009017 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 34720
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022904 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Number of data points in the train set: 17225, number of used features: 137
[LightGBM] [Info] Total Bins 34724
[LightGBM] [Info] Number of data points in the train set: 17225, number of used features: 137
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[Li

### 모델4 : TabNet

In [ ]:
import numpy as np
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.pipeline       import Pipeline
from imblearn.ensemble       import BalancedRandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection  import StratifiedKFold, cross_val_predict
from sklearn.metrics          import (
    classification_report,
    average_precision_score,
    recall_score,
    precision_score
)
from pytorch_tabnet.tab_model import TabNetClassifier
from torch.optim import Adam as TorchAdam

# 0) 준비된 데이터: df_tr, df_te, y_train, y_test
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
SM = BorderlineSMOTE(sampling_strategy=0.3, kind='borderline-1', random_state=42)

# 1) TabNet용 sklearn 래퍼 (에포크 늘리고 얼리스탑 patience 조정)
class TabNetSklearn(TabNetClassifier):
    def fit(self, X, y, **kwargs):
        X_np = X.values if hasattr(X, "values") else X
        y_np = y.values if hasattr(y, "values") else y
        super().fit(
            X_np, y_np,
            eval_set=[(X_np, y_np)],
            max_epochs=100,         # 최대 2000 에포크까지 학습
            patience=5,            # 50 에포크 동안 개선 없으면 얼리스탑
            batch_size=1024,
            virtual_batch_size=128,
            **kwargs
        )
        return self

    def predict_proba(self, X):
        X_np = X.values if hasattr(X, "values") else X
        return super().predict_proba(X_np)

# 2) 평가 함수 (변경 없음)
def evaluate(pipe, X_tr, y_tr, X_te, y_te, thresh=0.5):
    from joblib import parallel_backend
    with parallel_backend('threading', n_jobs=-1):
        y_val_proba = cross_val_predict(
            pipe, X_tr, y_tr,
            cv=cv, method='predict_proba', n_jobs=-1
        )[:,1]

    val_pr  = average_precision_score(y_tr, y_val_proba)
    y_val_pred = (y_val_proba >= thresh).astype(int)
    print("===== Validation (5-fold) =====")
    print(f"PR AUC   : {val_pr:.4f}")
    print(f"Recall   : {recall_score(y_tr, y_val_pred):.4f}")
    print(f"Precision: {precision_score(y_tr, y_val_pred):.4f}")
    print(classification_report(y_tr, y_val_pred, digits=4))

    pipe.fit(X_tr, y_tr)
    y_test_proba = pipe.predict_proba(X_te)[:,1]
    y_test_pred  = (y_test_proba >= thresh).astype(int)
    test_pr  = average_precision_score(y_te, y_test_proba)
    print("===== Test =====")
    print(f"PR AUC   : {test_pr:.4f}")
    print(f"Recall   : {recall_score(y_te, y_test_pred):.4f}")
    print(f"Precision: {precision_score(y_te, y_test_pred):.4f}")
    print(classification_report(y_te, y_test_pred, digits=4))

# 3) 파이프라인 정의 (변경 없음)
pipe_tabnet = Pipeline([
    ('smote',    SM),
    ('feat_sel', SelectFromModel(
        estimator=BalancedRandomForestClassifier(n_estimators=100, random_state=42),
        threshold=-np.inf, prefit=False
    )),
    ('clf',      TabNetSklearn(
                     n_d=16, n_a=16, n_steps=5,
                     gamma=1.3,
                     optimizer_fn=TorchAdam,
                     optimizer_params=dict(lr=2e-2),
                     mask_type='entmax'
                 ))
])

# 4) 실행
print("### TabNet ###")
evaluate(pipe_tabnet, df_tr, y_train, df_te, y_test, thresh=0.5)


### TabNet ###
epoch 0  | loss: 0.77415 | val_0_auc: 0.70003 |  0:00:03s
epoch 0  | loss: 0.7547  | val_0_auc: 0.72136 |  0:00:03s
epoch 0  | loss: 0.81041 | val_0_auc: 0.66594 |  0:00:03s
epoch 0  | loss: 0.78844 | val_0_auc: 0.71084 |  0:00:03s
epoch 0  | loss: 0.76948 | val_0_auc: 0.68426 |  0:00:03s
epoch 1  | loss: 0.52214 | val_0_auc: 0.80032 |  0:00:06s
epoch 1  | loss: 0.51072 | val_0_auc: 0.81617 |  0:00:06s
epoch 1  | loss: 0.52316 | val_0_auc: 0.84622 |  0:00:06s
epoch 1  | loss: 0.55084 | val_0_auc: 0.76153 |  0:00:06s
epoch 1  | loss: 0.52834 | val_0_auc: 0.81366 |  0:00:06s
epoch 2  | loss: 0.43461 | val_0_auc: 0.87031 |  0:00:09s
epoch 2  | loss: 0.43574 | val_0_auc: 0.86849 |  0:00:09s
epoch 2  | loss: 0.43571 | val_0_auc: 0.87291 |  0:00:10s
epoch 2  | loss: 0.41208 | val_0_auc: 0.88471 |  0:00:10s
epoch 2  | loss: 0.47274 | val_0_auc: 0.85918 |  0:00:10s
epoch 3  | loss: 0.38561 | val_0_auc: 0.88381 |  0:00:13s
epoch 3  | loss: 0.37301 | val_0_auc: 0.90743 |  0:00:13s

KeyboardInterrupt: 

epoch 60 | loss: 0.07131 | val_0_auc: 0.99766 |  0:03:11s
epoch 60 | loss: 0.04112 | val_0_auc: 0.99949 |  0:03:12s
epoch 60 | loss: 0.05249 | val_0_auc: 0.99882 |  0:03:12s


### 모델 5 : LSTM

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as data_utils
from sklearn.base import BaseEstimator, ClassifierMixin
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import (
    classification_report, average_precision_score,
    recall_score, precision_score
)
from joblib import parallel_backend

# ------------------------------
# 1. LSTMClassifier 정의
# ------------------------------
class LSTMClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, input_size, hidden_size=64, num_layers=1, epochs=10,
                 batch_size=64, lr=1e-3, device=None):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.epochs = epochs
        self.batch_size = batch_size
        self.lr = lr
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = None

    def _build_model(self):
        class LSTMNet(nn.Module):
            def __init__(self, input_size, hidden_size, num_layers):
                super().__init__()
                self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
                self.fc = nn.Linear(hidden_size, 1)

            def forward(self, x):
                out, _ = self.lstm(x)
                return torch.sigmoid(self.fc(out[:, -1, :]))

        return LSTMNet(self.input_size, self.hidden_size, self.num_layers).to(self.device)

    def fit(self, X, y):
        if hasattr(X, "values"):
            X = X.values
        if hasattr(y, "values"):
            y = y.values

        X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(1).to(self.device)
        y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1).to(self.device)

        dataset = data_utils.TensorDataset(X_tensor, y_tensor)
        loader = data_utils.DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

        self.model = self._build_model()
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        criterion = nn.BCELoss()

        self.model.train()
        for _ in range(self.epochs):
            for xb, yb in loader:
                optimizer.zero_grad()
                pred = self.model(xb)
                loss = criterion(pred, yb)
                loss.backward()
                optimizer.step()

        return self

    def predict_proba(self, X):
        if hasattr(X, "values"):
            X = X.values

        self.model.eval()
        X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(1).to(self.device)
        with torch.no_grad():
            probs = self.model(X_tensor).squeeze().cpu().numpy()
        return np.vstack([1 - probs, probs]).T

# ------------------------------
# 2. Stratified KFold & SMOTE
# ------------------------------
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
SM = BorderlineSMOTE(sampling_strategy=0.3, kind='borderline-1', random_state=42)

# ------------------------------
# 3. LSTM 파이프라인 정의
# ------------------------------
pipe_lstm_top50 = Pipeline([
    ('smote', SM),
    ('feat_sel', SelectFromModel(
        estimator=BalancedRandomForestClassifier(n_estimators=100, random_state=42),
        max_features=50,
        threshold=-np.inf,
        prefit=False
    )),
    ('clf', LSTMClassifier(input_size=50, epochs=10))
])

# ------------------------------
# 4. 평가 함수
# ------------------------------
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    average_precision_score, classification_report,
    precision_score, recall_score
)

def evaluate_manual_cv(pipe, X_tr, y_tr, X_te, y_te, thresh=0.5, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    val_probs = np.zeros(len(X_tr))
    val_preds = np.zeros(len(X_tr))

    for train_idx, val_idx in skf.split(X_tr, y_tr):
        X_train_fold, X_val_fold = X_tr.iloc[train_idx], X_tr.iloc[val_idx]
        y_train_fold, y_val_fold = y_tr.iloc[train_idx], y_tr.iloc[val_idx]

        pipe.fit(X_train_fold, y_train_fold)
        val_proba = pipe.predict_proba(X_val_fold)[:, 1]
        val_probs[val_idx] = val_proba
        val_preds[val_idx] = (val_proba >= thresh).astype(int)

    print("===== Validation (manual CV) =====")
    print(f"PR AUC   : {average_precision_score(y_tr, val_probs):.4f}")
    print(f"Recall   : {recall_score(y_tr, val_preds):.4f}")
    print(f"Precision: {precision_score(y_tr, val_preds):.4f}")
    print(classification_report(y_tr, val_preds, digits=4))

    pipe.fit(X_tr, y_tr)
    y_test_proba = pipe.predict_proba(X_te)[:, 1]
    y_test_pred = (y_test_proba >= thresh).astype(int)

    print("===== Test =====")
    print(f"PR AUC   : {average_precision_score(y_te, y_test_proba):.4f}")
    print(f"Recall   : {recall_score(y_te, y_test_pred):.4f}")
    print(f"Precision: {precision_score(y_te, y_test_pred):.4f}")
    print(classification_report(y_te, y_test_pred, digits=4))



In [ ]:
evaluate_manual_cv(pipe_lstm_top50, df_tr, y_train, df_te, y_test, thresh=0.5)

===== Validation (manual CV) =====
PR AUC   : 0.2435
Recall   : 0.3143
Precision: 0.2689
              precision    recall  f1-score   support

           0     0.9798    0.9749    0.9773     13115
           1     0.2689    0.3143    0.2898       385

    accuracy                         0.9561     13500
   macro avg     0.6243    0.6446    0.6336     13500
weighted avg     0.9595    0.9561    0.9577     13500

===== Test =====
PR AUC   : 0.3251
Recall   : 0.3854
Precision: 0.2984
              precision    recall  f1-score   support

           0     0.9819    0.9735    0.9776      3279
           1     0.2984    0.3854    0.3364        96

    accuracy                         0.9567      3375
   macro avg     0.6401    0.6794    0.6570      3375
weighted avg     0.9624    0.9567    0.9594      3375



### 모델 6 - FT-Transformer

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as data_utils
from sklearn.base import BaseEstimator, ClassifierMixin

# ------------------------------
# FTTransformerClassifier 정의
# ------------------------------
class FTTransformerClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, input_size, n_blocks=2, n_heads=4, d_token=64,
                 batch_size=64, epochs=10, lr=1e-3, device=None):
        self.input_size = input_size
        self.n_blocks = n_blocks
        self.n_heads = n_heads
        self.d_token = d_token
        self.batch_size = batch_size
        self.epochs = epochs
        self.lr = lr
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = None

    def _build_model(self):
        class TransformerBlock(nn.Module):
            def __init__(self, d_token, n_heads):
                super().__init__()
                self.attn = nn.MultiheadAttention(d_token, n_heads, batch_first=True)
                self.norm1 = nn.LayerNorm(d_token)
                self.ff = nn.Sequential(
                    nn.Linear(d_token, d_token * 4),
                    nn.ReLU(),
                    nn.Linear(d_token * 4, d_token)
                )
                self.norm2 = nn.LayerNorm(d_token)

            def forward(self, x):
                attn_out, _ = self.attn(x, x, x)
                x = self.norm1(x + attn_out)
                ff_out = self.ff(x)
                return self.norm2(x + ff_out)

        class FTTransformer(nn.Module):
            def __init__(self, input_size, d_token, n_heads, n_blocks):
                super().__init__()
                self.input_proj = nn.Linear(input_size, d_token)
                self.blocks = nn.Sequential(*[
                    TransformerBlock(d_token, n_heads) for _ in range(n_blocks)
                ])
                self.cls = nn.Linear(d_token, 1)

            def forward(self, x):
                x = self.input_proj(x)  # [B, 1, d_token]
                x = self.blocks(x)
                return torch.sigmoid(self.cls(x[:, 0, :]))

        return FTTransformer(self.input_size, self.d_token, self.n_heads, self.n_blocks).to(self.device)

    def fit(self, X, y):
        if hasattr(X, "values"):
            X = X.values
        if hasattr(y, "values"):
            y = y.values

        X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(1).to(self.device)
        y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1).to(self.device)

        dataset = data_utils.TensorDataset(X_tensor, y_tensor)
        loader = data_utils.DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

        self.model = self._build_model()
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        criterion = nn.BCELoss()

        self.model.train()
        for _ in range(self.epochs):
            for xb, yb in loader:
                optimizer.zero_grad()
                pred = self.model(xb)
                loss = criterion(pred, yb)
                loss.backward()
                optimizer.step()

        return self

    def predict_proba(self, X):
        if hasattr(X, "values"):
            X = X.values

        self.model.eval()
        X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(1).to(self.device)
        with torch.no_grad():
            probs = self.model(X_tensor).squeeze().cpu().numpy()
        return np.vstack([1 - probs, probs]).T


In [ ]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.feature_selection import SelectFromModel

# SMOTE 및 feature selection 설정은 기존과 동일
pipe_ft_top50 = Pipeline([
    ('smote', BorderlineSMOTE(sampling_strategy=0.3, kind='borderline-1', random_state=42)),
    ('feat_sel', SelectFromModel(
        estimator=BalancedRandomForestClassifier(n_estimators=100, random_state=42),
        max_features=50,
        threshold=-np.inf,
        prefit=False
    )),
    ('clf', FTTransformerClassifier(input_size=50, epochs=10))
])
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    classification_report, average_precision_score,
    precision_score, recall_score
)

def evaluate_manual_cv(pipe, X_tr, y_tr, X_te, y_te, thresh=0.5, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    val_probs = np.zeros(len(X_tr))
    val_preds = np.zeros(len(X_tr))

    for train_idx, val_idx in skf.split(X_tr, y_tr):
        X_train_fold = X_tr.iloc[train_idx]
        y_train_fold = y_tr.iloc[train_idx]
        X_val_fold = X_tr.iloc[val_idx]
        y_val_fold = y_tr.iloc[val_idx]

        pipe.fit(X_train_fold, y_train_fold)
        proba = pipe.predict_proba(X_val_fold)[:, 1]
        val_probs[val_idx] = proba
        val_preds[val_idx] = (proba >= thresh).astype(int)

    # Validation 성능 출력
    print("===== Validation (manual CV) =====")
    print(f"PR AUC   : {average_precision_score(y_tr, val_probs):.4f}")
    print(f"Recall   : {recall_score(y_tr, val_preds):.4f}")
    print(f"Precision: {precision_score(y_tr, val_preds):.4f}")
    print(f"F1 Score : {f1_score(y_tr, val_preds):.4f}")
    print(classification_report(y_tr, val_preds, digits=4))

    # 전체 학습 후 Test 성능
    pipe.fit(X_tr, y_tr)
    y_test_proba = pipe.predict_proba(X_te)[:, 1]
    y_test_pred = (y_test_proba >= thresh).astype(int)

    print("===== Test =====")
    print(f"PR AUC   : {average_precision_score(y_te, y_test_proba):.4f}")
    print(f"Recall   : {recall_score(y_te, y_test_pred):.4f}")
    print(f"Precision: {precision_score(y_te, y_test_pred):.4f}")
    print(f"F1 Score : {f1_score(y_te, y_test_pred):.4f}")
    print(classification_report(y_te, y_test_pred, digits=4))


In [ ]:
evaluate_manual_cv(pipe_ft_top50, df_tr, y_train, df_te, y_test, thresh=0.5)

===== Validation (manual CV) =====
PR AUC   : 0.3767
Recall   : 0.5365
Precision: 0.3674
F1 Score : 0.4361
              precision    recall  f1-score   support

           0     0.9821    0.9649    0.9734     16563
           1     0.3674    0.5365    0.4361       630

    accuracy                         0.9492     17193
   macro avg     0.6747    0.7507    0.7048     17193
weighted avg     0.9595    0.9492    0.9537     17193

===== Test =====
PR AUC   : 0.4037
Recall   : 0.5823
Precision: 0.3948
F1 Score : 0.4706
              precision    recall  f1-score   support

           0     0.9838    0.9660    0.9748      4141
           1     0.3948    0.5823    0.4706       158

    accuracy                         0.9518      4299
   macro avg     0.6893    0.7741    0.7227      4299
weighted avg     0.9621    0.9518    0.9562      4299



### 가장 높은 성능을 보이는 LGBM을 활용한 SHAP value 시각화

In [ ]:
import shap

# 1. LGBM 모델 가져온 후 학습
lgbm_model = pipe_lgbm_top50.named_steps['clf']
lgbm_model.fit(df_tr, y_train)

# 2. SHAP explainer 생성 (Tree 기반 모델 → TreeExplainer 사용)
explainer = shap.TreeExplainer(lgbm_model)

# 3. SHAP 값 계산 (큰 데이터면 샘플링)
X_sample = pd.DataFrame(df_tr, columns=df_tr.columns).sample(n=1000, random_state=42) \
    if len(df_tr) > 1000 else pd.DataFrame(df_tr, columns=df_tr.columns)
shap_exp = explainer(X_sample)

# SHAP 요약 그래프 (자동 다차원 처리)
shap.summary_plot(shap_exp.values, X_sample)